<a href="https://colab.research.google.com/github/yuriyasui/LLM-Prompting/blob/main/LLM_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title GCP Config

print("Installing Google Cloud Platform ......")
!pip install --upgrade google-cloud-aiplatform --quiet

project_id = "elaborate-leaf-417505"

print("There will be popup asking permission to access your Google credentials. Click Allow and follow subsequent steps to grant access")
from google.colab import auth as google_auth
google_auth.authenticate_user(project_id=project_id)
!gcloud config set project {project_id}
!gcloud config get-value project

import vertexai
from vertexai.preview.generative_models import GenerativeModel, GenerationConfig, Image, Content, Part

Installing Google Cloud Platform ......
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 25.4 MB/s eta 0:00:00
There will be popup asking permission to access your Google credentials. Click Allow and follow subsequent steps to grant access
Updated property [core/project].
elaborate-leaf-417505


In [ ]:
# @title Installing 3rd Party Modules

print("Installing ....")
!pip install -U langchain bs4 --quiet
import bs4
from langchain.document_loaders import WebBaseLoader

Installing ....
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 12.0 MB/s eta 0:00:00


In [ ]:
# @title Document Loading: Web Scrapping Multiple Issues

web_paths = (
    # "https://github.com/sympy/sympy/issues/26355",
    # "https://github.com/sympy/sympy/issues/26329",
    # "https://github.com/sympy/sympy/issues/26319",
    "https://www.sentinelone.com/blog/exit-sandman-how-sentinelone-deflects-apt-level-identity-security-risks/",
    )

context = ""
for path in web_paths:
    loader = WebBaseLoader(
    web_paths=(path,),
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(
            # class_=("d-block comment-body markdown-body js-comment-body title")
            class_=("entry-title", "entry-content")
            # class_=("entry-title")
        )
    },
)
    docs = loader.load()
    context += docs[0].page_content

print(context)

Exit Sandman | How SentinelOne Deflects APT-Level Identity Security Risks













Information theft and the number of data breaches rooted in identity-based risks are rising as attackers continue to exploit vulnerabilities and find ways to evade detection. This makes early detection one of the most critical pillars of defense across today’s attack surfaces. As identity-based threats continue to develop, organizations that focus on advanced detection and response can protect their data from skilled adversaries.
Advanced persistent threats (APTs) like Sandman, for example, have been observed using identity-based attacks to achieve initial access and lateral movement. These kinds of threat groups are not looking for ransom payments, meaning information theft is their most likely objective.
Increasing cases of information theft put organizations at risk of cyber espionage, financial loss, and brand damage. For organizations to counter such threats, early discovery is key. In this post

In [ ]:
# @title Generate response

#To do: Write your root prompt using Persona Task Context technique
root_prompt = """Summarize these issues"""
# root_prompt = """Does the following content inform about specific malware? If so, please explain what kind of malware it is talking about?"""

def send_to_gemini():
    model = GenerativeModel("gemini-pro")
    prompt = f"""{root_prompt}
    {context}
    """
    responses = model.generate_content(
        prompt,
        generation_config={
            "max_output_tokens": 4000,
            "temperature": 0.2,
            "top_p": 1,
            "top_k": 32
        },stream=True)
    data = ""
    for response in responses:
        data += response.text
    print(data)
    # return data
    return

send_to_gemini()

**Identity-Based Security Risks**

* Information theft and data breaches are increasing due to identity-based vulnerabilities.
* Advanced persistent threats (APTs) like Sandman use identity-based attacks for initial access and lateral movement.
* Organizations face risks of cyber espionage, financial loss, and brand damage from information theft.

**Case Study: Sandman APT**

* Sandman APT targeted telecommunication providers for espionage.
* They used credential theft and PtH techniques to infiltrate and move laterally.
* Detection and interruption prevented further attacks.

**SentinelOne's Identity Security Solutions**

* **Singularity Identity:**
    * Conceals credentials and AD objects from discovery.
    * Detects and alerts on credential-based attacks.
    * Deploys deceptive credentials to misdirect attackers.
* **Singularity Hologram:**
    * Provides enterprise-wide decoys to engage attackers.
    * Detects and alerts on PtH and other lateral movement techniques.
    * Recor